## 1. This patient may have sepsis
<p>Sepsis is a deadly syndrome where a patient has a severe infection that causes organ failure. The sooner septic patients are treated, the more likely they are to survive, but sepsis can be challenging to recognize. It may be possible to use hospital data to develop machine learning models that could flag patients who are likely to be septic. However, before we develop predictive algorithms, we need a reliable method to determine patients who are septic. One component of sepsis is a severe infection.</p>
<p>In this project, we will use two weeks of hospital electronic health record (EHR) data to find out which patients had a severe infection according to four criteria. We will look into the data to see if a doctor ordered a blood test to look for bacteria (a blood culture) and gave the patient a series of intervenous antibiotics. </p>
<p>Let's get started!</p>

In [333]:
# Load packages
# .... YOUR CODE FOR TASK 1 ....
#install.packages("data.table")
library(data.table)
# Read in the data
antibioticDT <- fread("C:\\Users\\Varun\\Downloads\\project medtour\\antibioticDT.csv")
##antibioticDT
#3head(antibioticDT,30)
antibioticDT[1:30]

# Look at the first 30 rows
# .... YOUR CODE FOR TASK 1 ....

patient_id,day_given,antibiotic_type,route
1,2,ciprofloxacin,IV
1,4,ciprofloxacin,IV
1,6,ciprofloxacin,IV
1,7,doxycycline,IV
1,9,doxycycline,IV
1,15,penicillin,IV
1,16,doxycycline,IV
1,18,ciprofloxacin,IV
8,1,doxycycline,PO
8,2,penicillin,IV


## 2. Which antibiotics are "new"?
<p>These data represent all drugs administered in a hospital over two weeks. Each row represents one time a patient was given an antibiotic. The variables include the patient identification number, the day the drug was administered, the name of the antibiotic, and how it was administered. For example, patient "8" received doxycycline by mouth on the first day of their stay.</p>
<p>We will identify patients with a serious infection using the following criteria. </p>
<p><strong>Criteria for Suspected Infection</strong><a href="https://www.ncbi.nlm.nih.gov/pubmed/28903154">*</a></p>
<ol>
<li>The patient receives antibiotics for a sequence of four days, with gaps of one day allowed.</li>
<li>The sequence must start with a new antibiotic, defined as an antibiotic type that was not given in the previous two days.</li>
<li>The sequence must start within two days of a blood culture.</li>
<li>There must be at least one intervenous (I.V.) antibiotic within the +/-2 day window.</li>
</ol>
<p>Let's start with the second item by finding which rows represent "new antibiotics". We will determine if each antibiotic was given to the patient in the prior two days. We'll visualize this task by looking at the data sorted by id, antibiotic type, and day.</p>

In [334]:
# Sort the data and examine the first 40 rows
# .... YOUR CODE FOR TASK 2 .....

setorder(antibioticDT,patient_id,antibiotic_type,day_given)
antibioticDT[1:40]
#data1=antibioticDT[,.(patient_id,antibiotic_type,day_given)]
#data1

# Use shift to calculate the last day a particular drug was administered
antibioticDT[ , last_administration_day := shift(day_given), 
  by = .(patient_id, antibiotic_type)]
#antibioticDT

# Calculate the number of days since the drug was last administered
antibioticDT[ , days_since_last_admin := day_given-last_administration_day] #shift(last_administration_day)]
#antibioticDT

# Create antibiotic_new with an initial value of one, then reset it to zero as needed
# .... YOUR CODE FOR TASK 2 .....
antibioticDT[,antibiotic_new:=(1)]
#antibioticDT
antibioticDT$antibiotic_new[antibioticDT$days_since_last_admin<=2]<-0
antibioticDT

patient_id,day_given,antibiotic_type,route
1,2,ciprofloxacin,IV
1,4,ciprofloxacin,IV
1,6,ciprofloxacin,IV
1,18,ciprofloxacin,IV
1,7,doxycycline,IV
1,9,doxycycline,IV
1,16,doxycycline,IV
1,15,penicillin,IV
8,1,doxycycline,PO
8,3,doxycycline,IV


patient_id,day_given,antibiotic_type,route,last_administration_day,days_since_last_admin,antibiotic_new
1,2,ciprofloxacin,IV,NA,NA,1
1,4,ciprofloxacin,IV,2,2,0
1,6,ciprofloxacin,IV,4,2,0
1,18,ciprofloxacin,IV,6,12,1
1,7,doxycycline,IV,NA,NA,1
1,9,doxycycline,IV,7,2,0
1,16,doxycycline,IV,9,7,1
1,15,penicillin,IV,NA,NA,1
8,1,doxycycline,PO,NA,NA,1
8,3,doxycycline,IV,1,2,0


## 3. Looking at the blood culture data
<p>Now let's look at blood culture data from the same two-week period in this hospital. These data are in <code>blood_cultureDT.csv</code>. Let's start by reading it into the workspace and having a look at a few rows. </p>
<p>Each row represents one blood culture and gives the patient's id and the day the blood culture test occurred. For example, patient "8" had a blood culture on the second day of their hospitalization and again on the thirteenth day. Notice that some patients from the antibiotic dataset are not in this dataset and vice versa. Some patients are in neither because they received neither antibiotics nor a blood culture.</p>

In [3]:
# Read in blood_cultureDT.csv
blood_cultureDT <- fread("blood_cultureDT.csv")
blood_cultureDT[1:30]
# Print the first 30 rows
# .... YOUR CODE FOR TASK 3 ....

patient_id,blood_culture_day
1,3
1,13
8,2
8,13
23,3
39,10
45,4
45,9
45,11
51,3


## 4. Combine the antibiotic data and the blood culture data
<p>To find which antibiotics were given close to a blood culture test, we need to combine the drug administration data with the blood culture data. We'll keep only patients that are still candidates for infection&mdash;only those in both data sets.</p>
<p>A challenge with the data is that some patients had blood cultures on several different days. For each of those days, we will see if there is a sequence of antibiotic days close to them. To accomplish this, in the merge we will match each blood culture to each antibiotic day.</p>
<p>After sorting the data following the merge, you will see that each patient's antibiotic sequence repeats for each blood culture day. This repetition allows us to look at each blood culture day and check if it is associated with a qualifying sequence of antibiotics.</p>

In [4]:
# Merge antibioticDT with blood_cultureDT
combinedDT <-merge(antibioticDT,blood_cultureDT,by.antibioticDT="patient_id",by.blood_cultureDT="patient_id") 
#combinedDT

# Sort by patient_id, blood_culture_day, day_given, and antibiotic_type
# .... YOUR CODE FOR TASK 4 .....
setorder(combinedDT,patient_id,blood_culture_day,day_given,antibiotic_type)
#combinedDT

# Print and examine the first 30 rows
# .... YOUR CODE FOR TASK 4 .....
combinedDT[1:30]

patient_id,day_given,antibiotic_type,route,last_administration_day,days_since_last_admin,antibiotic_new,blood_culture_day
1,2,ciprofloxacin,IV,NA,NA,1,3
1,4,ciprofloxacin,IV,2,2,0,3
1,6,ciprofloxacin,IV,4,2,0,3
1,7,doxycycline,IV,NA,NA,1,3
1,9,doxycycline,IV,7,2,0,3
1,15,penicillin,IV,NA,NA,1,3
1,16,doxycycline,IV,9,7,1,3
1,18,ciprofloxacin,IV,6,12,1,3
1,2,ciprofloxacin,IV,NA,NA,1,13
1,4,ciprofloxacin,IV,2,2,0,13


## 5. Determine whether each row is in-window
<p>Now that we have the antibiotic and blood culture data combined, we can test each drug administration against each blood culture to see if it's "in the window."</p>

In [5]:
# Make a new variable called drug_in_bcx_window
# ....YOUR CODE FOR TASK 5 ....
combinedDT[,drug_in_bcx_window:=ifelse(last_administration_day-blood_culture_day<=2,1,0)]
combinedDT


patient_id,day_given,antibiotic_type,route,last_administration_day,days_since_last_admin,antibiotic_new,blood_culture_day,drug_in_bcx_window
1,2,ciprofloxacin,IV,NA,NA,1,3,NA
1,4,ciprofloxacin,IV,2,2,0,3,1
1,6,ciprofloxacin,IV,4,2,0,3,1
1,7,doxycycline,IV,NA,NA,1,3,NA
1,9,doxycycline,IV,7,2,0,3,0
1,15,penicillin,IV,NA,NA,1,3,NA
1,16,doxycycline,IV,9,7,1,3,0
1,18,ciprofloxacin,IV,6,12,1,3,0
1,2,ciprofloxacin,IV,NA,NA,1,13,NA
1,4,ciprofloxacin,IV,2,2,0,13,1


## 6. Check the I.V. requirement
<p>Now let's look at the fourth item in the criteria. </p>
<p><strong>Criteria for Suspected Infection</strong><a href="https://www.ncbi.nlm.nih.gov/pubmed/28903154">*</a></p>
<ol>
<li>The patient receives antibiotics for a sequence of four days, with gaps of one day allowed.</li>
<li>The sequence must start with a new antibiotic, defined as an antibiotic type that was not given in the previous two days.</li>
<li>The sequence must start within two days of a blood culture.</li>
<li><em>There must be at least one intervenous (I.V.) antibiotic within the +/-2 day window.</em></li>
</ol>

In [336]:
# Create a variable indicating if there was at least one I.V. drug given in the window
# .... YOUR CODE FOR TASK 6 ....
combinedDT[,any_iv_in_bcx_window:=ifelse(antibiotic_new==0 & drug_in_bcx_window==1,"TRUE","FALSE")]

# Exclude rows in which the blood_culture_day does not have any I.V. drugs in window 
# .... YOUR CODE FOR TASK 6 ....
combinedDT<-combinedDT[combinedDT$any_iv_in_bcx_window== "TRUE"]
combinedDT

patient_id,day_given,antibiotic_type,route,last_administration_day,days_since_last_admin,antibiotic_new,blood_culture_day,drug_in_bcx_window,any_iv_in_bcx_window,day_of_first_new_abx_in_window
1,4,ciprofloxacin,IV,2,2,0,3,1,TRUE,4
1,6,ciprofloxacin,IV,4,2,0,3,1,TRUE,4
1,4,ciprofloxacin,IV,2,2,0,13,1,TRUE,4
1,6,ciprofloxacin,IV,4,2,0,13,1,TRUE,4
1,9,doxycycline,IV,7,2,0,13,1,TRUE,9
8,3,doxycycline,IV,1,2,0,2,1,TRUE,3
8,3,doxycycline,IV,1,2,0,13,1,TRUE,3
23,3,doxycycline,IV,1,2,0,3,1,TRUE,3
23,4,doxycycline,IV,3,1,0,3,1,TRUE,3
23,5,ciprofloxacin,PO,3,2,0,3,1,TRUE,5


## 7. Find the first day of possible sequences
<p>We're getting close! Let's review the criteria again.</p>
<p><strong>Criteria for Suspected Infection</strong><a href="https://www.ncbi.nlm.nih.gov/pubmed/28903154">*</a></p>
<ol>
<li>The patient receives antibiotics for a sequence of four days, with gaps of one day allowed.</li>
<li>The sequence must start with a new antibiotic, defined as an antibiotic type that was not given in the previous two days.</li>
<li>The sequence must start within two days of a blood culture.</li>
<li>There must be at least one intervenous (I.V.) antibiotic within the +/-2 day window.</li>
</ol>
<p>Let's assess the first criterion by finding the first day of possible 4-day qualifying sequences.    </p>

In [7]:
# Create a new variable called day_of_first_new_abx_in_window
# .... YOUR CODE FOR TASK 7 ....
setorder(combinedDT,patient_id,blood_culture_day,day_given,antibiotic_type)
#combinedDT
combinedDT[,day_of_first_new_abx_in_window:=day_given[1],by=.(blood_culture_day,antibiotic_type,patient_id)]
combinedDT
# Remove rows where the day is before this first qualifying day
# .... YOUR CODE FOR TASK 7 ....
#combinedDT[day_of_first_new_abx_in_window<day_of_first_new_abx_in_window[1],by=.(antibiotic_type,blood_culture_day)]

patient_id,day_given,antibiotic_type,route,last_administration_day,days_since_last_admin,antibiotic_new,blood_culture_day,drug_in_bcx_window,any_iv_in_bcx_window,day_of_first_new_abx_in_window
1,4,ciprofloxacin,IV,2,2,0,3,1,TRUE,4
1,6,ciprofloxacin,IV,4,2,0,3,1,TRUE,4
1,4,ciprofloxacin,IV,2,2,0,13,1,TRUE,4
1,6,ciprofloxacin,IV,4,2,0,13,1,TRUE,4
1,9,doxycycline,IV,7,2,0,13,1,TRUE,9
8,3,doxycycline,IV,1,2,0,2,1,TRUE,3
8,3,doxycycline,IV,1,2,0,13,1,TRUE,3
23,3,doxycycline,IV,1,2,0,3,1,TRUE,3
23,4,doxycycline,IV,3,1,0,3,1,TRUE,3
23,5,ciprofloxacin,PO,3,2,0,3,1,TRUE,5


## 8. Simplify the data
<p>The first criterion is: <em>The patient receives antibiotics for a sequence of four days, with gaps of one day allowed.</em></p>
<p>We've pinned down the first day of possible sequences in the previous task. Now we have to check for four-day sequences. We don't need the drug type (name); we need the days the antibiotics were administered.</p>

In [160]:
# Create a new data.table containing only patient_id, blood_culture_day, and day_given
simplified_data <-data.frame(combinedDT$patient_id,combinedDT$blood_culture_day,combinedDT$day_given)
simplified_data
nrow(simplified_data)
#2129
# Remove duplicate rows
#.... YOUR CODE FOR TASK 8 ....
simplified_data<-unique(simplified_data,by="combinedDT.day_given")
simplified_data

combinedDT.patient_id,combinedDT.blood_culture_day,combinedDT.day_given
1,3,4
1,3,6
1,13,4
1,13,6
1,13,9
8,2,3
8,13,3
23,3,3
23,3,4
23,3,5


[1] 2129

,combinedDT.patient_id,combinedDT.blood_culture_day,combinedDT.day_given
1,1,3,4
2,1,3,6
3,1,13,4
4,1,13,6
5,1,13,9
6,8,2,3
7,8,13,3
8,23,3,3
9,23,3,4
10,23,3,5


## 9. Extract first four rows for each blood culture
<p>To check for four-day sequences, let's pull out the first four days (rows) for each patient/blood culture combination. Some patients will have less than four antibiotic days. We'll remove them first.</p>

In [337]:
# Count the antibiotic days within each patient/blood culture day combination
# .... YOUR CODE FOR TASK 9 ....
 ## Remove blood culture days with less than four rows
simplified_data<-as.data.table(simplified_data)
#simplified_data
#class(simplified_data)
simplified_data[,num_antibiotic_days:=.N,.(combinedDT.blood_culture_day ,combinedDT.patient_id)]
#simplified_data
#nrow(simplified_data)2129
simplified_data<-simplified_data[simplified_data$num_antibiotic_days>=4]
#simplified_data
nrow(simplified_data)
#1015

# Select the first four days for each blood culture
first_four_days <- simplified_data[,.SD[1:4],.(combinedDT.blood_culture_day,combinedDT.patient_id)] 
first_four_days
nrow(first_four_days)

[1] 1015

combinedDT.blood_culture_day,combinedDT.patient_id,combinedDT.day_given,num_antibiotic_days
3,23,3,4
3,23,4,4
3,23,5,4
3,23,6,4
6,164,1,6
6,164,2,6
6,164,4,6
6,164,6,6
7,164,1,6
7,164,2,6


[1] 656

## 10. Consecutive sequence
<p>Now we need to check whether each four-day sequence qualifies by having no gaps of more than one day.</p>
<!--"Patient receives antibiotics for a sequence of 4 days, with gaps of 1 day allowed."-->

In [274]:
# Make the indicator for consecutive sequence
# .... YOUR CODE FOR TASK 9 ....
#class(first_four_days)
first_four_days<-as.data.table(first_four_days)
first_four_days[,lagday:= shift(combinedDT.day_given),.(combinedDT.patient_id,combinedDT.blood_culture_day)]
#first_four_days
first_four_days[,four_in_seq:=combinedDT.day_given-lagday]
first_four_days$four_in_seq<-as.numeric(first_four_days$four_in_seq<=1)
###it is 1 if true, else 0
first_four_days


combinedDT.blood_culture_day,combinedDT.patient_id,combinedDT.day_given,num_antibiotic_days,lagday,four_in_seq
3,23,3,4,NA,NA
3,23,4,4,3,1
3,23,5,4,4,1
3,23,6,4,5,1
6,164,1,6,NA,NA
6,164,2,6,1,1
6,164,4,6,2,0
6,164,6,6,4,0
7,164,1,6,NA,NA
7,164,2,6,1,1


## 11. Select the patients who meet criteria
<p>A patient would meet the criteria if any of their blood cultures were accompanied by a qualifying sequence of antibiotics. Now that we've determined which each blood culture qualify let's select the patients who meet the criteria.</p>

In [320]:
# Select the rows which have four_in_seq equal to 1
suspected_infection <- first_four_days[four_in_seq==1 ]
suspected_infection=subset(suspected_infection,select=-c(combinedDT.blood_culture_day,combinedDT.day_given,num_antibiotic_days,lagday,four_in_seq))
#suspected_infection
nrow(suspected_infection)
#376
# Retain only the patient_id column
# .... YOUR CODE FOR TASK 9 ....
#keeps<-c("combinedDT.patient_id")
#suspected_infection=suspected_infection[keeps]
#suspected_infection<-suspected_infection[combinedDT.patient_id]
#suspected_infection
# Remove duplicates
# .... YOUR CODE FOR TASK 9 ....
suspected_infection<-unique(suspected_infection)
#suspected_infection
nrow(suspected_infection)
#102

# Make an infection indicator
# .... YOUR CODE FOR TASK 9 ....
suspected_infection[,infection:=1]
infection_flag<-suspected_infection
infection_flag<-as.data.table(infection_flag)
infection_flag
nrow(infection_flag)

[1] 367

[1] 102

combinedDT.patient_id,infection
23,1
164,1
200,1
206,1
237,1
294,1
311,1
372,1
402,1
415,1


[1] 102

## 12. Find the prevalence of sepsis
<p>In this project, we used two EHR datasets to flag patients who were suspected of having a serious infection. We also got a <code>data.table</code> workout!</p>
<p>So far, we've been looking at records of all antibiotics administered and blood cultures that occurred over two weeks at a particular hospital. However, not all patients who were hospitalized over this period are represented in <code>combinedDT</code> because not all of them took antibiotics or had blood culture tests. We have to read in and merge the rest of the patient information to see what percentage of patients at the hospital might have had a serious infection.</p>

In [341]:
# Read in "all_patients.csv"
all_patientsDT <- fread("C:\\Users\\Varun\\Downloads\\project medtour\\all_patients.csv")
nrow(all_patientsDT)
#890
#all_patientsDT

# Merge this with the infection flag data
all_patientsDT <- merge(all_patientsDT,infection_flag,by.x="patient_id",by.y="combinedDT.patient_id")
all_patientsDT
nrow(all_patientsDT)
#102
#
# Set any missing values of the infection flag to 0
# .... YOUR CODE FOR TASK 12 ....

# Calculate the percentage of patients who met the criteria for presumed infection
ans  <- (102/890)*100
ans
#11.406%


[1] 890

patient_id,infection
23,1
164,1
200,1
206,1
237,1
294,1
311,1
372,1
402,1
415,1


[1] 102

[1] 11.46067

[1] 102